<a href="https://colab.research.google.com/github/Ihien/ChatBot/blob/main/Projet_Fin_Etudes_Data_Science_Intensive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div align="center">
  <img src="https://dit.sn/wp-content/uploads/2023/10/Logo-1.png" alt="Logo Université" width="350"/>
  <h1></h1>
  <h1>Dakar Institute of Technology</h1>
  <h3>Data Science Department</h3>
</div>

<hr>

<div align="center">
  <h1>Projet de Fin d'Étude</h1>
  <em>Pour l'obtention du :</em>
  <br>Certificat Data Science Intensive<br>
  <h4>Thème :</h4>
  <h2>Mise en Place d'un ChatBot Basé sur LLM Pour Répondre aux Questions Usuelles des Clients</h2>
</div>

<br>
<br>

<div align="center">
  <div style="display: flex; justify-content: space-around;">
    <div style="text-align: left;">
      <strong>Présenté par :</strong><br>
      Isidore HIEN<br>
      <em>Matricule : [Votre Matricule]</em>
    </div>
    <div style="text-align: left;">
      <strong>Sous la direction de :</strong><br>
      Mouhamadou Naby DIA<br>
      <em>NLP ENgineer</em>
    </div>
  </div>
</div>

<br>
<br>
<br>

<div align="center">
  <p>Année Académique : 2025-2026</p>
</div>

---
### Sommaire
1.  [Introduction](#introduction)
2.  [Activation du GPU](https://colab.research.google.com/drive/16TqNaEYSplpgJ6R1tiXSrlsxnx7X-X15#scrollTo=02JX67cZtVAQ&line=5&uniqifier=1)
3.  [Modélisation et Entraînement](#modelisation)
4.  [Résultats et Évaluation](#resultats)
5.  [Conclusion](#conclusion)

---

# Introduction

Dans un contexte économique de plus en plus concurrentiel, la qualité et la réactivité du service client sont devenues des facteurs de différenciation majeurs pour les entreprises. Le centre d'appels, en tant que principal point de contact, se trouve au cœur de cette dynamique. Cependant, il est souvent confronté à des défis de taille : un volume d'appels élevé, des temps d'attente prolongés pour les clients, et une charge de travail répétitive pour les agents, qui doivent répondre continuellement aux mêmes questions fréquentes. Cette situation engendre non seulement une augmentation des coûts opérationnels, mais limite également la capacité des agents à se concentrer sur des problématiques complexes à plus forte valeur ajoutée.

Face à cette problématique, les avancées récentes dans le domaine de l'intelligence artificielle, et plus particulièrement des grands modèles de langage (LLM), offrent des opportunités de transformation sans précédent. Ces technologies permettent de créer des agents conversationnels (chatbots) capables de comprendre et de traiter le langage naturel avec une fluidité et une pertinence remarquables.

Ce projet vise à concevoir et mettre en œuvre un chatbot intelligent basé sur un LLM open source, destiné à automatiser la gestion des questions usuelles au sein d'un centre d'appels. En s'appuyant sur la technique de **Génération Augmentée par Récupération (RAG)**, notre solution ne se contentera pas de fournir des réponses génériques ; elle puisera ses informations directement dans une base de connaissance interne et sécurisée, garantissant ainsi des réponses précises, fiables et parfaitement adaptées au contexte de l'entreprise.

Les principaux **objectifs** de ce projet sont les suivants :
* **Développer** un agent conversationnel robuste en utilisant un LLM open source (`Mistral`) hébergé localement.
* **Implémenter** un système de qualification systématique pour identifier chaque interlocuteur (nom, secteur d'activité, contact) avant de traiter sa demande.
* **Connecter** le chatbot à une base de connaissance privée composée de documents d'entreprise (PDF, TXT, etc.).
* **Assurer** la fiabilité des réponses en forçant le modèle à se baser exclusivement sur les documents fournis, minimisant ainsi les risques d'hallucination.
* **Déployer** une démonstration fonctionnelle de l'application dotée d'une interface utilisateur interactive.

Ce rapport détaillera l'ensemble de la méthodologie adoptée, de la configuration de l'environnement technique à la préparation de la base de connaissance, en passant par l'implémentation de la logique conversationnelle et, enfin, le déploiement d'un prototype fonctionnel.

#1. Activation du GPU
Pour commencer, il faut activer le GPU afin de faciliter l'exécution du projet.
1.   Allez dans le menu Exécution -> Modifier le type d'exécution.
2.   Dans le menu déroulant "Accélérateur matériel", sélectionnez GPU T4.
3.   Cliquez sur "Enregistrer".



# 2. Créer et Téléverser vos Documents

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#3. Installation d'Ollama dans le Notebook

In [2]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


#4. Lancement d'Ollama en Arrière-Plan
Ollama doit tourner comme un service. Cette commande le lance en arrière-plan pour que nous puissions l'utiliser dans les cellules suivantes.

In [3]:
import os
import asyncio
# Commande pour lancer le serveur en arrière-plan
command = "nohup ollama serve > ollama.log 2>&1 &"
# Exécuter la commande
os.system(command)
async def wait_for_server():
    await asyncio.sleep(5)

print("Serveur Ollama démarré en arrière-plan.")

Serveur Ollama démarré en arrière-plan.


#5. Installation des Librairies Python
Nous installons les librairies Python nécessaires ainsi que toutes leurs dépendances. Ces librairies faciliteront l'installation du modèle LLM plus tard.

In [4]:
!pip install langchain langchain_community chromadb sentence-transformers msoffcrypto-tool unstructured "unstructured[pdf]" "unstructured[docx]" "unstructured[xlsx]" "unstructured[pptx]" --upgrade chainlit pyngrok -U langchain-ollama langchain-huggingface

# 6. Téléchargement du Modèle LLM
Ici nous téléchargeons le modèle LLM. Nous choisissons mistral pour sa puissance et son rapport performance/taille.

In [5]:
!ollama pull mistral

# 6. Configuration du ChatBot

## 6.1. Importations de librairies

In [6]:
import os
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

## 6.2. Configuration des Modèles

In [7]:
EMBEDDING_MODEL_NAME = "BAAI/bge-m3"
OLLAMA_MODEL_NAME = "mistral"

llm = ChatOllama(
    model=OLLAMA_MODEL_NAME, # ex: "mistral" ou "llama3.1:8b"
    temperature=0.2,         # Très basse pour des réponses factuelles
    top_p=0.8,
    top_k=50                 # On peut réduire top_k pour être plus strict
)

print(f"Modèle LLM '{OLLAMA_MODEL_NAME}' chargé avec une température de {llm.temperature}.")
embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    model_kwargs={'device': 'cuda'},        # Force l'utilisation du GPU
    encode_kwargs={'normalize_embeddings': True} # Active la normalisation
)

Modèle LLM 'mistral' chargé avec une température de 0.2.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## 6.3. Chargement et Traitement des Documents
Nous utilisons un chargeur qui prend tous les fichiers d'un dossier. Il reconnaît automatiquement les types de fichiers (PDF, TXT, etc.).

Les documents chargés sont divisés en plus petits morceaux (chunks). Cette étape est cruciale pour la performance du RAG.

In [8]:
!apt-get install poppler-utils -y
!apt-get update && apt-get install tesseract-ocr-fra -y
!apt-get install -y ocrmypdf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.10).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Readi

In [9]:
KNOWLEDGE_BASE_DIR = "/content/drive/Othercomputers/Mon Laptop Asus/Data_Science/Projet_Fin_Etude/knowledge_base"

print(f"Chargement des documents depuis le dossier : {KNOWLEDGE_BASE_DIR}")

# chargeur
loader = DirectoryLoader(KNOWLEDGE_BASE_DIR, glob="**/*.*", show_progress=True, loader_kwargs={"languages": ["fra"]})
documents = loader.load()

# Divise les documents chargés en plus petits morceaux (chunks)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)

print(f"{len(documents)} document(s) chargé(s) et divisé(s) en {len(splits)} morceaux.")

Chargement des documents depuis le dossier : /content/drive/Othercomputers/Mon Laptop Asus/Data_Science/Projet_Fin_Etude/knowledge_base


100%|██████████| 6/6 [03:10<00:00, 31.71s/it]

6 document(s) chargé(s) et divisé(s) en 109 morceaux.


# 7. Création de la base de données vectorielle
La **base vectorielle** est essentielle pour relier un **LLM** aux documents internes d’une organisation.  
Elle transforme ces documents en **embeddings** (vecteurs) permettant une **recherche sémantique** efficace.  

Son rôle principal est d’alimenter le chatbot dans une architecture **RAG (Retrieval-Augmented Generation)** :  
1. Indexer et vectoriser les documents internes.  
2. Stocker les vecteurs dans une base spécialisée (Pinecone, FAISS, Weaviate, etc.).  
3. Lors d’une question, retrouver les passages pertinents.  
4. Fournir ces passages au LLM pour générer une réponse adaptée.  

**Avantages clés :**  
- Accès rapide aux données internes (catalogue, fiches de produits, FAQ).  
- Recherche basée sur le sens et non sur les mots-clés.  
- Mise à jour facile du savoir du chatbot.  
- Réduction des erreurs et hallucinations du modèle.  
- Contrôle et personnalisation du contenu des réponses.  

La base vectorielle transforme un LLM générique en **chatbot intelligent, spécialisé et fiable**, capable de répondre avec précision à partir de vos propres documents.

In [10]:
# Création de la Base de Données Vectorielle (à partir des documents)
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)
retriever = vectorstore.as_retriever()
print("Base de connaissance vectorisée et prête.")


# --- Chaîne de Traitement RAG ---
system_prompt_template = """
Tu es un assistant virtuel expert du service client de notre entreprise.
Ton ton doit être professionnel, clair et serviable.
Réponds à la question de l'utilisateur en te basant exclusivement sur le contexte suivant fourni.
Si l'information n'est pas dans le contexte, réponds poliment que tu ne disposes pas de cette information.

Contexte fourni :
{context}

Question de l'utilisateur :
{question}
"""
prompt = ChatPromptTemplate.from_template(system_prompt_template)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("Chaîne RAG prête !")

Base de connaissance vectorisée et prête.
Chaîne RAG prête !


# 8. Logique de Conversation et de Qualification

## 8.1. Création du Gestionnaire de Conversation

Cette étape est cruciale dans le projet car elle permet de structurer le dialogue entre l’utilisateur et le chatbot.  
Le **gestionnaire de conversation** ne se limite pas à répondre aux questions : il **collecte d’abord les informations clés** (nom, secteur d’activité, lieu d’exercice, contact), ce qui permet de :

1. **Qualifier l’utilisateur** : avant de fournir une réponse, le chatbot s’assure d’avoir les données minimales nécessaires pour personnaliser l’échange.  
2. **Améliorer la pertinence des réponses** : en disposant du contexte utilisateur, le chatbot peut adapter son langage et ses conseils.  
3. **Faciliter la traçabilité** : les informations collectées peuvent être stockées ou utilisées pour des analyses ultérieures (CRM, support client, suivi des demandes).  
4. **Assurer une expérience utilisateur fluide** : la progression est guidée par des questions ciblées, évitant que l’utilisateur ait à tout fournir d’un seul coup.  
5. **Optimiser l’usage du RAG** : une fois la qualification terminée, la question de fond est transmise à la chaîne RAG pour générer une réponse enrichie et documentée.  

Cette section agit comme un **chef d’orchestre** du projet : elle garantit que le chatbot est capable de **comprendre qui interagit avec lui, dans quel contexte, et avec quelle question**, avant de mobiliser les documents vectorisés pour produire une réponse fiable et personnalisée.

In [11]:
class ConversationManager:
    def __init__(self, llm, rag_chain):
        self.llm = llm
        self.rag_chain = rag_chain
        # C'est ici que nous définissons les informations que nous voulons collecter.
        self.required_info = {
            "nom": None,
            "secteur_activité": None,
            "lieu_exercice": None,
            "contact": None,
        }
        self.user_question = None
        self.qualification_prompt = ChatPromptTemplate.from_template(
            """Extrais les informations suivantes du texte de l'utilisateur : nom, secteur_activité, lieu_exercice, contact, et la question de fond qu'il pose.
            Réponds UNIQUEMENT avec les informations trouvées, une par ligne, comme ceci :
            nom: [nom trouvé ou "non trouvé"]
            secteur_activité: [secteur trouvé ou "non trouvé"]
            lieu_exercice: [lieu trouvé ou "non trouvé"]
            contact: [contact trouvé ou "non trouvé"]
            question: [question de fond trouvée ou "non trouvé"]

            Texte de l'utilisateur : "{user_input}"
            """
        )
        self.qualification_chain = self.qualification_prompt | self.llm | StrOutputParser()

    def is_qualified(self):
        """Vérifie si toutes les informations requises ont été collectées."""
        return all(value is not None for value in self.required_info.values())

    def get_next_question(self):
        """Détermine la prochaine question à poser."""
        # Personnalisez les questions posées par le chatbot.
        if self.required_info["nom"] is None:
            return "Bonjour ! Pour mieux vous assister, pourriez-vous commencer par vous présenter (votre nom et prénom(s)) ?"
        if self.required_info["secteur_activité"] is None:
            return "Merci. Dans quel secteur d'activité travaillez-vous ?"
        if self.required_info["lieu_exercice"] is None:
            return "Parfait. Où est situé le lieu d'exercice de votre activité (ville/région) ?"
        if self.required_info["contact"] is None:
            return "Nous y sommes presque. Quel est votre contact (email ou numéro de téléphone) ?"
        return None

    def process_message(self, user_input):
        """Traite le message de l'utilisateur et retourne la réponse du chatbot."""
        # Si la qualification n'est pas terminée
        if not self.is_qualified():
            # On utilise le LLM pour extraire les informations de la réponse de l'utilisateur
            extracted_text = self.qualification_chain.invoke({"user_input": user_input})

            # Mise à jour des informations collectées
            for line in extracted_text.split('\n'):
                if ':' in line:
                    key, value = line.split(':', 1)
                    key = key.strip()
                    value = value.strip()
                    if key in self.required_info and value != "non trouvé":
                        self.required_info[key] = value
                    if key == "question" and value != "non trouvé":
                        self.user_question = value

            # Si on est maintenant qualifié ET qu'on a une question
            if self.is_qualified():
                print("--- QUALIFICATION TERMINÉE ---")
                print("Informations collectées :", self.required_info)
                if self.user_question:
                    response = f"Merci pour ces informations. Concernant votre question '{self.user_question}' :\n"
                    response += self.rag_chain.invoke(self.user_question)
                    return response
                else:
                    return "Merci pour ces informations. Comment puis-je vous aider maintenant ?"
            else:
                # Sinon, on pose la prochaine question
                return self.get_next_question()

        # Si la qualification est déjà terminée, on répond directement
        else:
            return self.rag_chain.invoke(user_input)

# Initialisation du gestionnaire
chatbot = ConversationManager(llm, rag_chain)
print("Gestionnaire de conversation prêt !")

Gestionnaire de conversation prêt !


# 8.2. Simulations

In [12]:
entree_utilisateur = "Bonjour, Qui es-tu?"
chatbot.process_message(entree_utilisateur)

--- QUALIFICATION TERMINÉE ---
Informations collectées : {'nom': 'Non trouvé', 'secteur_activité': 'Non trouvé', 'lieu_exercice': 'Non trouvé', 'contact': 'Non trouvé'}


"Merci pour ces informations. Concernant votre question 'Qui es-tu?' :\n Je suis un assistant virtuel expert du service client de notre entreprise. Mon rôle est d'aider les clients à trouver des informations et des solutions en utilisant le contexte fourni dans le cadre de notre projet actuel."

In [13]:
entree_utilisateur = "Bonjour"

reponse_chatbot = chatbot.process_message(entree_utilisateur)
print(f"🤖 Chatbot: {reponse_chatbot}\n")

🤖 Chatbot:  Bonjour !

Nous avons des projets d'installation de points de correspondance et les moyens de paiement via notre application mobile APV2, ainsi que l'ouverture d'autres agences. Nos produits de crédit sont conçus pour être simples, faciles et adaptés aux besoins du client. Nous avons également des produits d'épargne pour garantir à nos clients un avenir meilleur.

Pour plus d'informations sur nos services, je vous invite à consulter notre catalogue de produits 2025 en ligne. Si vous avez des questions spécifiques ou si vous souhaitez discuter de votre situation particulière, n'hésitez pas à me contacter.

Nous sommes au service de nos partenaires avec dynamisme et efficacité, et nous tenons nos promesses. Nous savons créer la confiance, récompenser et motiver nos équipes, être juste, équitable, transparent et accueillir les décisions difficiles avec sagesse.

Je suis à votre disposition pour vous aider.



# 9. Création de l'application

## 9.1. Ecriture du script complet de l'application

In [14]:
%%writefile app.py

import os
import chainlit as cl
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Configuration des Modèles
EMBEDDING_MODEL_NAME = "BAAI/bge-m3"
OLLAMA_MODEL_NAME = "mistral"

llm = ChatOllama(
    model=OLLAMA_MODEL_NAME,
    temperature=0.2,
    top_p=0.8,
    top_k=50
)

embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': True}
)

# Chargement et Préparation de la Base de Connaissance
KNOWLEDGE_BASE_DIR = "/content/drive/Othercomputers/Mon Laptop Asus/Data_Science/Projet_Fin_Etude/knowledge_base"

loader = DirectoryLoader(KNOWLEDGE_BASE_DIR, glob="**/*.*", show_progress=True)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)
retriever = vectorstore.as_retriever()

# Création de la Chaîne RAG
system_prompt_template = """
Tu es un assistant virtuel expert du service client de Baobab Burkina.
Ton ton doit être professionnel, clair et serviable.
Réponds à la question de l'utilisateur en te basant exclusivement sur le contexte suivant fourni.
Si l'information n'est pas dans le contexte, réponds poliment que tu ne disposes pas de cette information.

Contexte fourni :
{context}

Question de l'utilisateur :
{question}
"""
prompt = ChatPromptTemplate.from_template(system_prompt_template)
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Logique de Conversation
class ConversationManager:
    def __init__(self, llm, rag_chain):
        self.llm = llm
        self.rag_chain = rag_chain
        self.required_info = {
            "nom": None,
            "secteur_activité": None,
            "lieu_exercice": None,
            "contact": None,
        }
        self.user_question = None
        self.qualification_prompt = ChatPromptTemplate.from_template(
            """Extrais les informations suivantes du texte de l'utilisateur : nom, secteur_activité, lieu_exercice, contact, et la question de fond qu'il pose.
            Réponds UNIQUEMENT avec les informations trouvées, une par ligne, comme ceci :
            nom: [nom trouvé ou "non trouvé"]
            secteur_activité: [secteur trouvé ou "non trouvé"]
            lieu_exercice: [lieu trouvé ou "non trouvé"]
            contact: [contact trouvé ou "non trouvé"]
            question: [question de fond trouvée ou "non trouvé"]

            Texte de l'utilisateur : "{user_input}"
            """
        )
        self.qualification_chain = self.qualification_prompt | self.llm | StrOutputParser()

    def is_qualified(self):
        return all(value is not None for value in self.required_info.values())

    def get_next_question(self):
        if self.required_info["nom"] is None:
            return "Pour mieux vous assister, pourriez-vous commencer par vous présenter (votre nom et prénom(s)) ?"
        if self.required_info["secteur_activité"] is None:
            return "Merci. Dans quel secteur d'activité travaillez-vous ?"
        if self.required_info["lieu_exercice"] is None:
            return "Parfait. Où est situé le lieu d'exercice de votre activité (ville/région) ?"
        if self.required_info["contact"] is None:
            return "Nous y sommes presque. Quel est votre contact (email ou numéro de téléphone) ?"
        return None

    def process_message(self, user_input):
        if not self.is_qualified():
            extracted_text = self.qualification_chain.invoke({"user_input": user_input})
            for line in extracted_text.split('\n'):
                if ':' in line:
                    key, value = line.split(':', 1)
                    key = key.strip()
                    value = value.strip()
                    if key in self.required_info and value.lower() != "non trouvé":
                        self.required_info[key] = value
                    if key == "question" and value.lower() != "non trouvé" and not self.user_question:
                        self.user_question = value
            if self.is_qualified():
                if self.user_question:
                    response = f"Merci pour toutes ces informations. Concernant votre question '{self.user_question}' :\n"
                    response += self.rag_chain.invoke(self.user_question)
                    return response
                else:
                    return "Merci pour ces informations. Comment puis-je vous aider maintenant ?"
            else:
                return self.get_next_question()
        else:
            return self.rag_chain.invoke(user_input)

# Logique d'Interface avec Chainlit
@cl.on_chat_start
async def start_chat():
    """
    Cette fonction est appelée au début de chaque nouvelle conversation.
    """
    # On initialise le gestionnaire de conversation et on le stocke dans la session
    chatbot = ConversationManager(llm, rag_chain)
    cl.user_session.set("chatbot", chatbot)

    # On envoie un message de bienvenue général à l'utilisateur.
    await cl.Message(
        content="Bonjour, \nJe suis Yaaba Bot, l'assistant virtuel de Baobab Burkina. \nComment puis-je vous aider aujourd'hui ?"
    ).send()

@cl.on_message
async def main(message: cl.Message):
    """
    Cette fonction est appelée à chaque fois que l'utilisateur envoie un message.
    """
    # Toute la logique passe par ici.

    # On récupère le gestionnaire de conversation de la session
    chatbot = cl.user_session.get("chatbot")

    # On traite le message de l'utilisateur avec notre logique complète
    # La méthode process_message va gérer à la fois la qualification et la réponse.
    response = chatbot.process_message(message.content)

    # On envoie la réponse du chatbot
    await cl.Message(content=response).send()

Overwriting app.py


## 9.2. Configuration de l'interface Chainlit

In [15]:
#!mkdir -p .chainlit

In [16]:
#%%writefile .chainlit/config.toml
# [project]
# public = true
# database = "local"   # ou "cloud"
# enable_telemetry = true
# session_timeout = 3600

# [ui]
# Nom affiché en haut de l’interface
# name = "Assistant Baobab"

# Logo (optionnel)
# logo = "https://baobab.com/wp-content/uploads/2023/08/Logo.svg"

# Thème (⚠️ l’option default_theme est dépréciée → maintenant : "theme")
#theme = "light"   # valeurs possibles : "light" ou "dark"

# 10. Lancement de la démo

In [17]:
from pyngrok import ngrok
import os
from google.colab import userdata

# Obtenir l'authtoken ngrok à partir de Colab secrets
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')

# Authentifier ngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Lancer le tunnel ngrok sur le port 8000 (port par défaut de Chainlit)
public_url = ngrok.connect(8000)
print(f"La démo est accessible publiquement à cette adresse : {public_url}")

# Lancer l'application Chainlit en arrière-plan
# Le "-h" est pour "headless", ce qui évite d'ouvrir une interface dans Colab
os.system("chainlit run app.py -h &")

La démo est accessible publiquement à cette adresse : NgrokTunnel: "https://ca6015ac8b7e.ngrok-free.app" -> "http://localhost:8000"


0

In [18]:
# Version de débogage pour voir les logs
#!chainlit run app.py

In [19]:
# Supprime le fichier de configuration incompatible
#!rm /content/.chainlit/config.toml